In [2]:
import openai
import re
import json
import time
import requests
import os

In [3]:
# Path to the txt file containing the API key
api_key_file = 'openaikey.txt'

In [4]:
# Read the API key from the file
with open(api_key_file, 'r') as file:
    api_key = file.read().strip()

In [5]:
# Set the environment variable
os.environ["OPENAI_API_KEY"] = api_key

# Initialize OpenAI with the environment variable
openai.api_key = os.getenv("OPENAI_API_KEY")

In [6]:
# Function to clean the lyrics
def clean_lyrics(lyrics):
    cleaned_lyrics = re.sub(r'[^a-zA-Z0-9\s]', '', lyrics)  # Remove special characters
    cleaned_lyrics = re.sub(r'\n+', ' ', cleaned_lyrics)  # Replace multiple newlines with a space
    cleaned_lyrics = cleaned_lyrics.strip()  # Remove leading/trailing whitespace
    return cleaned_lyrics

In [7]:
# Function to fetch the lyrics of a song using the Lyrics.ovh API
def get_lyrics(artist, title):
    url = f"https://api.lyrics.ovh/v1/{artist}/{title}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data.get('lyrics', 'Lyrics not found')
    except requests.exceptions.HTTPError as errh:
        print(f"HTTP Error: {errh}")
    except requests.exceptions.ConnectionError as errc:
        print(f"Connection Error: {errc}")
    except requests.exceptions.Timeout as errt:
        print(f"Timeout Error: {errt}")
    except requests.exceptions.RequestException as err:
        print(f"Something went wrong: {err}")

In [8]:
# Function to generate short explanation of lyrics
def generate_explanation(song, lyrics):
    messages = [
        {"role": "system", "content": "You are an expert on song Taylor Swift's song lyrics and their meanings."},
        {"role": "user", "content": f"Explain the meaning of the song '{song}' by Taylor Swift. Lyrics: {lyrics}"}
    ]
    
    try:
        # chat/completions endpoint for the chat-based model
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=150,
            temperature=0.7
        )
        explanation = response.choices[0].message.content.strip()
        return explanation
    except Exception as e:
        print(f"Error generating explanation for {song}: {e}")
        return None

In [9]:
# Function to save the prompt-completion pair as JSONL
def save_single_pair_as_jsonl(pair, filename):
    with open(filename, 'a') as f:
        json.dump(pair, f)
        f.write('\n')

In [10]:
# Main loop to continuously process songs
def process_songs(filename='lyrics.jsonl'):
    artist = "Taylor Swift"
    
    while True:
        song = input("Enter the name of the song (or type 'exit' to stop): ")
        
        if song.lower() == 'exit':
            print("Exiting...")
            break

        # Fetch lyrics
        lyrics = get_lyrics(artist, song)
        if not lyrics or lyrics == 'Lyrics not found':
            print(f"Lyrics not found for song: {song}")
            continue

        # Clean lyrics
        cleaned_lyrics = clean_lyrics(lyrics)
        
        # Generate explanation using GPT-3
        explanation = generate_explanation(song, cleaned_lyrics)
        
        if explanation:
            # Create the prompt-completion pair
            prompt_completion_pair = {
                "prompt": f"Explain the meaning of the song '{song}' by Taylor Swift.",
                "completion": explanation
            }
            
            # Save the pair as a JSONL entry
            save_single_pair_as_jsonl(prompt_completion_pair, filename)
            print(f"Processed and saved explanation for song: {song}")
            
            # Pause to respect rate limits
            time.sleep(1)

In [11]:
# Run the main loop and generate lyrics
process_songs()

Processed and saved explanation for song: loml
Exiting...
